In [2]:
!pip install rdflib
!pip install PyDrive

You should consider upgrading via the '/Users/sofianagavkina/.virtualenvs/itmo-nlp/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/sofianagavkina/.virtualenvs/itmo-nlp/bin/python -m pip install --upgrade pip' command.


In [108]:
from rdflib import URIRef, BNode, Literal, Namespace, Graph
from rdflib.namespace import Namespace, NamespaceManager
from rdflib.plugins import sparql
from rdflib.namespace import RDF, RDFS, VOID
from rdflib.serializer import Serializer
import json
from typing import List, Dict
import requests
import re
import time

In [11]:
# create empty graph
graph = Graph()
graph.parse('last_ver.owl', format="turtle")
# print(graph.serialize(format='n3'))

<Graph identifier=N5b197f96b41e4c2caef80412c978eaf7 (<class 'rdflib.graph.Graph'>)>

In [105]:
# Вопрос: Найти популярную альтернативу конкретной машине по габаритам
qres = graph.query("""SELECT ?m2 ?man
                    WHERE { 
                        values ?car { <http://www.semanticweb.org/sofia/ontologies/2021/3/cars#FIESTA_HATCHBACK_5_DOORS_2017> }
                            ?car        :clearence    ?c .
                            ?s :clearence    ?c2 .
                            ?car        :width        ?w .
                            ?car        :length       ?l .
                            ?car        :height       ?h .
                            ?car        :isOfModel    ?m .
                            ?s :clearence    ?c2 .
                            ?s :width        ?w2 .
                            ?s :length       ?l2 .
                            ?s :height       ?h2 .
                            ?s        :isOfModel    ?m2 .
                            ?m2 :createdBy ?man
                            FILTER(?c2 < ?c + 5) .
                            FILTER(?c2 > ?c - 5) .
                            FILTER(?h2 < ?h + 50) .
                            FILTER(?h2 > ?h - 50) .
                            FILTER(?l2 < ?l + 200) .
                            FILTER(?l2 > ?l - 200) .
                            FILTER(?w2 < ?w + 50) .
                            FILTER(?w2 > ?w - 50) .
                            FILTER(?m != ?m2) .
                        }
                        GROUP BY ?m2
                        """)


for row in qres:
    print(row[1].split('#')[1], row[0].split('#')[1])

CITROEN C4
HONDA CIVIC
HYUNDAI I30
KIA CEED
CHEVROLET CORSA
TOYOTA COROLLA
VOLKSWAGEN GOLF
HYUNDAI I20
CHEVROLET AVEO
TOYOTA AURIS
PEUGEOT 308
DACIA LOGAN
FIAT STILO


In [89]:
# Вопрос: Какие редкие машины доступны на рынке?
qres = graph.query("""SELECT ?m ?man (sum(?count) AS ?Total)
                    WHERE { 
                        ?car        :isOfModel    ?m .
                        ?car        :availabilityCount    ?count .
                        ?m :createdBy ?man
                        FILTER(?count < 5) .
                        }
                        GROUP BY ?m 
                        HAVING (sum(?count) < 5)
                        """)


for row in qres:
    print(row[1].split('#')[1], row[0].split('#')[1], row[2])

SUZUKI MR_WAGON 3
OPEL COMMODORE 2
OPEL P4 2
AUDI Q8 4
PEUGEOT BIPPER 1
DODGE DAYTONA 4
FIAT SEICENTO 3
NISSAN 200SX 1
DAIHATSU MOVE_CONTE 3
ALFA_ROMEO 146 4
HONDA CROSSTOUR 3
HYUNDAI I10 3
DAEWOO DAMAS 3
CHRYSLER LHS 4
FORD BRONCO 4
CHEVROLET BEL_AIR 2
NISSAN XTERRA 4
HONDA EDIX 4
SUZUKI FORENZA 4
DAIHATSU PYZAR 3
FIAT REGATA 3
TOYOTA BREVIS 1
CITROEN XM 3
OPEL MERIVA 4
FORD BRONCO_II 3
OPEL TIGRA 4
VOLKSWAGEN TYPE_2 4
TOYOTA BLADE 2
TOYOTA AQUA 4
FORD MODEL_A 2
FIAT FREEMONT 1
KIA X_TREK 2
OPEL VECTRA_OPC 2
AUDI S7 4
DAIHATSU MOVE_CANBUS 1
CHEVROLET VAN 4
HYUNDAI MARCIA 1
SUZUKI SPACIA 4
HONDA ELEMENT 4
HONDA MDX 2
OPEL GRANDLAND_X 2
CHEVROLET MASTER 1
FORD FUSION 4
NISSAN PRIMASTAR 3
HONDA HR_V 4
CHRYSLER SARATOGA 2
RENAULT 11 3
TOYOTA COROLLA_SPACIO 1
SUZUKI PALETTE 2
CHEVROLET ASTRO 4
HYUNDAI SCOUPE 3
DODGE AVENGER 2
VOLKSWAGEN KARMANN_GHIA 2
SUZUKI ESTEEM 4
HYUNDAI IX55 4
HONDA NSX 1
NISSAN PINO 3
NISSAN ROOX 2
FORD PUMA 4
DAEWOO LEGANZA 3
CHRYSLER IMPERIAL_CROWN 1
FORD MONDEO_ST

In [107]:
# Вопрос: Какие есть редкие машины с  низким расходом топлива?
qres = graph.query("""SELECT ?m ?man ?fuel
                    WHERE { 
                        ?car        :isOfModel    ?m .
                        ?car        :availabilityCount    ?count .
                        ?car        :fuelConsumption    ?fuel .
                        ?m :createdBy ?man
                        FILTER(?count < 5) .
                        FILTER(?fuel < 5) .
                        }
                        GROUP BY ?m 
                        HAVING (sum(?count) < 5)
                        """)


for row in qres:
    print(row[1].split('#')[1], row[0].split('#')[1], row[2])

TOYOTA AQUA 2
CITROEN C3 4
VOLVO S80 4
ALFA_ROMEO MITO 4
TOYOTA AYGO 4
CITROEN C4_CACTUS 4
VOLVO V70 4
CITROEN C4_SPACETOURER 3
AUDI A3 4
AUDI A1 4
VOLVO V90 4


In [104]:
# Вопрос: Сколько редких машин произведено в конкретной стране?
qres = graph.query("""SELECT ?m ?man (sum(?count) AS ?Total)
                    WHERE { 
                        ?car        :isOfModel    ?m .
                        ?car        :availabilityCount    ?count .
                        ?car        :fuelConsumption    ?fuel .
                        ?m :createdBy ?man.
                        ?man :country "Germany".
                        FILTER(?count < 5) .
                        }
                        GROUP BY ?m 
                        HAVING (sum(?count) < 5)
                        """)


for row in qres:
    print(row[1].split('#')[1], row[0].split('#')[1], row[2])

AUDI E_TRON 4
AUDI Q8 4
AUDI S5 1
OPEL GT 3


In [115]:
# generate VOID
from rdflib.void import generateVoID
q = generateVoID(graph)
print(q)
q[0].serialize(destination='cars_with_void.rdf')

(<Graph identifier=Na7facb2916d743818f04221c75906cf9 (<class 'rdflib.graph.Graph'>)>, rdflib.term.URIRef('http://example.org/Dataset'))
